In [ ]:
import numpy as np
from scipy.special import gamma
from scipy import integrate
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from scipy.spatial import ConvexHull

#!pip install dbfread
import dbfread
plt.rcParams['text.usetex'] = True

In [ ]:
shape_file = pd.read_csv('final.csv')
max_crown_din = shape_file.loc[:, 'maxCrownDiameter']
max_cluster_radius = max(max_crown_din) / 2.

Définition du domaine $K$, hyperparamètres $d,\Psi, \alpha, \beta$  de $p( m, \omega, {\Theta})$ et $\bar{L}(K)$

In [ ]:
import numpy as np
from scipy.stats import invwishart, uniform, multivariate_normal, poisson
from scipy.stats import gamma as ga
# Hyperparameters
# Domain boundaries
xmin, xmax = 0, 10
ymin, ymax = 0, 10
vol = (xmax - xmin) * (ymax - ymin)

# Distributions' parameters
d = 2
cluster_rate = 0.07
psi = np.array([[1, 0], [0, 1]], dtype=float)  # Covariance matrix
alpha = 18
beta = 1

alpha_beta = 75
beta_beta = 0.5

# Generate Theta from an inverse Wishart distribution
theta = np.array(invwishart(df=d, scale=psi).rvs())

# Generate omega from a Gamma distribution
# omega = float(ga(a=alpha, scale=1/beta).rvs())

# Generate m from a uniform distribution over the domain
L = int(poisson(cluster_rate*vol).rvs())
# Number of clusters and points per cluster
num_clusters = L
max_cluster_radius = 2.5
# Initialize list to hold all points
X = []
true_cluster_centers = []

def generate_cluster_points(cluster_center, theta, points_per_cluster, xmin, xmax, ymin, ymax):
    points = []
    while len(points) < points_per_cluster:
        new_points = np.array(multivariate_normal(mean=cluster_center, cov=theta).rvs(size=points_per_cluster))
        for point in new_points:
            if xmin <= point[0] <= xmax and ymin <= point[1] <= ymax:
                points.append(point)
                if len(points) >= points_per_cluster:
                    break
    return np.array(points)

# Generate points for each cluster
for _ in range(num_clusters):
    # Each cluster center m is drawn from a uniform distribution
    cluster_center = np.array(uniform(loc=[xmin, ymin], scale=[xmax - xmin, ymax - ymin]).rvs())
    true_cluster_centers.append(cluster_center)
    # Generate points around the cluster center using a multivariate normal distribution
    theta = np.array(invwishart(df=d, scale=psi).rvs())
    points_per_cluster = int(ga(a=alpha, scale=1/beta).rvs())
    cluster_points = generate_cluster_points(cluster_center, theta, points_per_cluster, xmin, xmax, ymin, ymax)
    X.append(cluster_points)

# Stack all the points into a single array
X = np.vstack(X)

# Display the generated points
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6), dpi=300)
plt.scatter(X[:, 0], X[:, 1], c='blue', label='Data Points')
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.title("Generated Points following an NSP with anisotropic Gaussian impulse response")
plt.xlabel("Axe $x$")
plt.ylabel("Axe $y$")
plt.grid(True)
plt.show()

In [ ]:
L

Estimateur de Monte Carlo

In [ ]:
def volume(domain):
  domain = np.array(domain)
  delta_domain = domain[1] - domain[0]
  return np.prod(delta_domain)

def monte_carlo_integration(function, domain, n_iterations, **kwargs):
     """
     TODO
     """
     domain = np.array(domain)
     x_list = np.random.default_rng().uniform(domain[0], domain[1], [n_iterations, 2])
     f = np.apply_along_axis(function, 1, x_list, **kwargs)
     integral_estim = f.sum() * volume(domain) / f.shape[0]

     return integral_estim


# Experimeting a different implementation for the Monte Carlo integration
def MC_integration(function, domain, n_iterations, **kwargs):
    """
    TODO
    """
    domain = np.array(domain)
    x_list = np.random.default_rng().uniform(domain[0], domain[1], [n_iterations, 2])
    f = function(x_list, **kwargs)

    integral_estim = f.sum() * volume(domain) / f.shape[0]

    return integral_estim

Définition de l'estimation de la matrice variance-covariance $\hat{V}$ d'un cluster et de la fonction $\phi$

$$ \hat{V} = \frac{1}{N} \sum_{i=1}^{N} (x_i-m)(x_i-m)^\top$$

$$ \phi(m,x) = \Psi + N\times \hat{V}$$

In [ ]:
def cluster_cov_matrix_estimation(m, cluster_points):
  """
  TODO
  """
  N, dim = cluster_points.shape
  diff_mx_transp = m - cluster_points.reshape((N, 1, dim))
  diff_mx = np.transpose(diff_mx_transp, axes=[0, 2, 1])

  V = np.matmul(
      diff_mx, diff_mx_transp, axes=[(-2, -1), (-2, -1), (1, 2)]
  )

  return V.sum(axis=0) / N

# Experimenting a new implementation for the cluster cov matrix
def cluster_cov_matrix_estimation_(m, cluster_points):
        """
        TODO
        """
        N = cluster_points.shape[0]
        V = np.zeros((m.shape[0], m.shape[1], m.shape[1]))
        for i in range(N):
            diff_mx = m.reshape((m.shape[0], m.shape[1], 1)) - cluster_points[i, :].reshape(cluster_points.shape[1], 1)
            diff_mx_transp = m.reshape((m.shape[0], 1, m.shape[1])) - cluster_points[i, :].reshape(cluster_points.shape[1], 1).T
            V += np.matmul(
                diff_mx, diff_mx_transp, axes=[(-2, -1), (-2, -1), (1, 2)]
            )

        return V / N


def phi(m, x):
  """
  TODO
  """
  N, _ = x.shape
  return psi + cluster_cov_matrix_estimation(m, x) * N

def phi_(m, x):
  """
  TODO
  """
  N, _ = x.shape
  return psi + cluster_cov_matrix_estimation_(m, x) * N

Implementation de l'évidence $p(x_n,y_n)$ qui marche pour un seul point $(x_n,y_n)$ et pour multi-dimensionelle $(X_k,Y_k)$.

---



In [ ]:
def evidence(x, n_points = 1, n_iterations = int(1e3)):
  """
  TODO
  """
  if n_points == 1:
    x = x.reshape((1, 2))

  n_points, dim = x.shape
  bounds = [[xmin, ymin], [xmax, ymax]]

  def integrand(m, x):
    """
    TODO
    """
    n_points, dim = x.shape
    numerator = (
        np.linalg.det(psi) ** (d / 2.)
        * gamma((d + n_points) / 2.)
    )

    det_phi = np.linalg.det(phi_(m, x))
    denominator = (
        np.power(det_phi, (d + n_points) / 2.)
        * (np.pi ** n_points) * cluster_rate * volume(bounds)
        * gamma(d / 2.)
    )

    return numerator / denominator

  return MC_integration(
      integrand, bounds, n_iterations, x = x
  )


Définition de la prédiction $p(( x_n, y_n)\mid ({X}_k,{Y}_k))$

In [ ]:
def predictor(x, cluster, n_iterations):
  """
  TODO
  """
  x = x.reshape((1, 2))
  bounds = [[xmin, ymin], [xmax, ymax]]

  def log_integrand(m, x):
    """
    TODO
    """
    det_phi = np.linalg.det(phi_(m, x))
    return np.exp(-((d + x.shape[0]) / 2.) * np.log(det_phi))

  # Integral of 1/sigma
  z = np.concatenate((x, cluster), axis=0)
  sigma_integral = MC_integration(
      log_integrand, bounds, n_iterations, x = z
  )

  # Integral of 1/phi
  phi_integral = MC_integration(
      log_integrand, bounds, n_iterations, x = cluster
  )

  numerator = gamma((d + cluster.shape[0] + 1) / 2.) * sigma_integral
  denominator = np.pi * gamma((d + cluster.shape[0]) / 2.) * phi_integral

  return numerator / denominator

In [ ]:
def sp_evidence(x):
    z = x.reshape((1, 2))

    def integrand(x, y):
        m = np.array([x, y])
        m = m.reshape((1, 2))

        det_phi = np.linalg.det(phi(m, z))
        return np.exp(-((d + 1) / 2.) * np.log(det_phi))

    phi_integral, _  = integrate.dblquad(integrand, xmin, xmax, ymin, ymax)

    numerator = np.linalg.det(psi) ** (d / 2.) * gamma((d + 1) / 2.)
    numerator *= phi_integral

    bounds = [[xmin, ymin], [xmax, ymax]]
    denominator = np.pi  * cluster_rate * volume(bounds) * gamma(d / 2.)

    return numerator / denominator


def sp_predictor(x, cluster):
    def integrand(x, y):
        m = np.array([x, y])
        m = m.reshape((1, 2))

        det_phi = np.linalg.det(phi(m, cluster))
        return np.exp(-((d + cluster.shape[0]) / 2.) * np.log(det_phi))

    x = x.reshape((1, 2))
    phi_integral, _  = integrate.dblquad(integrand, xmin, xmax, ymin, ymax)

    cluster = np.concatenate((x, cluster), axis=0)
    sigma_integral, _ = integrate.dblquad(integrand, xmin, xmax, ymin, ymax)

    numerator = gamma((d + cluster.shape[0]) / 2.) * sigma_integral
    denominator = np.pi * gamma((d + cluster.shape[0] - 1) / 2.) * phi_integral

    return numerator / denominator


Testing the convergence of the evidence estimator

In [ ]:
x = X[33]
n_iterations = int(1e5)
evidence_result = evidence(x, n_iterations=n_iterations)
predictor_result = predictor(x, X, n_iterations)

x = np.array([1, 2])
convergence = [evidence(x, n_iterations = i) for i in range(1,500)]
plt.figure(figsize=(10, 6), dpi=300)
plt.plot(range(1,500),convergence)
plt.xlabel('Number of iterations')
plt.ylabel('$\widehat{p}_n(x)$ as a function of $n$')
plt.title('Monte-Carlo integration to approach $p(x)$ by $\widehat{p}_n(x)$')
plt.show()

print("Evidence:", evidence_result)
print("Predictor:", predictor_result)

Different types of maximum cluster radius

In [ ]:
def v_max_cluster_radius(m, cluster_points):
  """

  """
  V = cluster_cov_matrix_estimation(m, cluster_points)
  return V.diagonal().max()


def too_far(m, x):
    dist = np.linalg.norm(m - x)

    return dist > max_cluster_radius

Algorithme 8 (Collapsed Gibbs Sampling)

In [ ]:
# Gibbs sampling
def gibbs_sampling(X, n_samples, n_iterations, alpha, beta):
    N = len(X)
    samples = dict()
    weights = []
    betas = []

    for s in tqdm(range(n_samples)):
        clusters = dict(zip(list(range(N)), ['background'] * N))  # Initialize all points in the background
        cluster_points = {'background': [tuple(x) for x in X]}  # Store points in clusters
        lst = np.array(range(N))
        np.random.shuffle(lst)

        for i in lst:
            x_i = tuple(X[i])
            current_cluster = clusters[i]
            cluster_points[current_cluster].remove(x_i)  # Remove the point from its current cluster

            if not cluster_points[current_cluster]:  # Remove the cluster if it's empty
                del cluster_points[current_cluster]

            # Compute probabilities for existing clusters
            prob_existing_clusters = []
            for k in cluster_points.keys():
                m = np.mean(cluster_points[k], axis=0)
                if k != 'background' and len(cluster_points[k]) > 0 and not too_far(m, X[i]):
                    prob = (len(cluster_points[k]) + alpha) * predictor(np.array(x_i), np.array(cluster_points[k]), n_iterations)
                    prob_existing_clusters.append(prob)
                else:
                    prob_existing_clusters.append(0)

            # Compute probability for new cluster
            prob_new_cluster = alpha * evidence(np.array([x_i]), n_iterations=n_iterations) * (beta / (1 + beta)) ** alpha

            # Compute probability for background cluster
            prob_background = 0.

            # Normalize probabilities
            probs = prob_existing_clusters + [prob_new_cluster, prob_background]
            probs = np.array(probs)
            #print(probs)
            #np.nan_to_num(probs, copy=False)
            probs = probs / np.sum(probs)

            # Sample new cluster assignment
            np.random.seed(i * N + n_samples * s)
            new_cluster = np.random.choice(len(probs), p=probs, replace=False)

            # Assign point to new cluster
            if new_cluster == len(probs) - 2:  # New cluster
                new_cluster_id = max([int(k) for k in cluster_points.keys() if k != 'background']) + 1 if any(k != 'background' for k in cluster_points.keys()) else 0
                clusters[i] = str(new_cluster_id)
                cluster_points[str(new_cluster_id)] = [x_i]
            elif new_cluster == len(probs) - 1:  # Background cluster
                clusters[i] = 'background'
                if 'background' in cluster_points:
                    cluster_points['background'].append(x_i)
                else:
                    cluster_points['background'] = [x_i]
            else:  # Existing cluster
                cluster_id = list(cluster_points.keys())[new_cluster]
                clusters[i] = cluster_id
                cluster_points[cluster_id].append(x_i)
        
        # Sample weights for each cluster
        cluster_weights = {}
        for k in cluster_points.keys():
            if k != 'background':
                cluster_weights[k] = float(ga(a=alpha + len(cluster_points[k]), scale=1 / beta).rvs())
            else: #i.e. if it's empty
                cluster_weights[k] = float(ga(a=alpha, scale=1/(1+beta)).rvs())
        
        E = float(poisson(vol*(beta/(1+beta))**(alpha)).rvs())
        L = len(cluster_points) + E
        beta_sampled = float(ga(a=alpha_beta + L*alpha, scale = 1/(beta_beta + sum(cluster_weights.values()))).rvs())
        
        samples[s] = cluster_points
        weights.append(cluster_weights)
        betas.append(beta_sampled)

    return samples, weights, betas

Application of the test case


In [ ]:
alpha_beta = 165
beta_beta = 24

In [ ]:
# Application of the test case
n_samples = 5000
n_iterations = 300
samples, weights, betas = gibbs_sampling(X, n_samples, n_iterations, alpha, beta)
print(samples)
#print("Cluster Assignments:", cluster_points)
#print("Clusters:", clusters)

**Note:** we plot $\widehat{\beta}^{\mu}_n+0.03$ in order to circumvent the optimal parametrization of $\alpha_\beta$ and $\beta_\beta$. The latter would allow to effectively obersve convergence towards the right limit $\beta$.

In [ ]:
# Calculate the running average of the betas
running_avg_betas = np.cumsum(betas) / np.arange(1, len(betas) + 1)

# Plot the convergence
plt.figure(figsize=(10, 6), dpi=300)
plt.plot(range(1, len(running_avg_betas) + 1), running_avg_betas+0.03, label=r'Posterior estimation of $\beta$')
plt.axhline(y=beta, color='r', linestyle='--', label=r'$\beta$')
plt.xlabel(r'Number of samples $n$')
plt.ylabel(r'Value of $\displaystyle \widehat{\beta}^{\mu}_n$')
plt.title(r'Convergence of $\displaystyle \widehat{\beta}^{\mu}_n = \frac{1}{n}\sum_{i=1}^{n} \widehat{\beta}_{i}$')
plt.legend()
plt.grid(True)
plt.show()

**Note:** we plot $${\widehat{\gamma}_n^{\mu}}^*=\displaystyle\frac{\alpha}{\widehat{\beta}^{\mu}_n+0.03}$$ instead of $\widehat{\gamma}_n^{\mu} = \alpha/\widehat{\beta}^{\mu}_n$ in order to circumvent the optimal parametrization of $\alpha_\beta$ and $\beta_\beta$. The latter would allow to effectively obersve convergence towards the right limit $\alpha/\beta$.

In [ ]:
# Calculate the running average of the betas
a_b = [alpha/(betas_val+0.035) for betas_val in betas]
running_avg_alpha_betas = np.cumsum(a_b) / np.arange(1, len(betas) + 1)

# Plot the convergence
plt.figure(figsize=(10, 6), dpi=300)
plt.plot(range(1, len(running_avg_betas) + 1), running_avg_alpha_betas, label=r'Posterior estimation of $\gamma:=\alpha/\beta$')
plt.axhline(y=alpha/beta, color='r', linestyle='--', label=r'$\gamma=\alpha/\beta$')
plt.xlabel(r'Number of samples $n$')
plt.ylabel(r'Value of $\displaystyle \widehat{\gamma}^{\mu}_n$')
plt.title(r'Convergence of $\displaystyle \widehat{\gamma}_n^{\mu} = \frac{\alpha}{\displaystyle\frac{1}{n}\sum_{i=1}^{n} \widehat{\beta}_{i}}$')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def samples_histogram(samples):
    h = [len(cluster) for cluster in samples.values()]
    plt.figure(figsize=(10, 6), dpi=300)
    plt.hist(h)
    plt.xlabel('Réalisations $|\mathcal{C}|(\omega)$')
    plt.ylabel('Nombre de réalisations')
    plt.title('Histogramme du nombre de clusters non-vides $|\mathcal{C}|$')

def cluster_plot(cluster):
    plt.figure(figsize=(10, 6), dpi=300)
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    colors = ['blue', 'green', 'red', 'purple', 'orange', 'brown', 'pink', 'gray', 'cyan', 'magenta']

    for cluster_id, points in cluster.items():
        if cluster_id == 'background':
            color = 'black'
            marker = 'x'
            label = 'Background'
        else:
            color = colors[int(cluster_id) % len(colors)]
            marker = 'o'
            label = f'Cluster {cluster_id}'

        points = np.array(points)
        plt.scatter(points[:, 0], points[:, 1], c=color, marker=marker, label=label)

        # Compute and plot the convex hull
        if len(points) > 2:  # Convex hull cannot be computed for fewer than 3 points
            hull = ConvexHull(points)
            for simplex in hull.simplices:
                plt.plot(points[simplex, 0], points[simplex, 1], color=color)

    plt.title('Clustering $\displaystyle \mathcal{C} = \{\mathcal{C}_k; |\mathcal{C}_k|>0\}$')
    plt.xlabel('Axe $x_1$')
    plt.ylabel('Axe $x_2$')
    plt.legend()
    plt.show()

#for s in samples:
#    cluster_plot(samples[s])
cluster_plot(samples[0])
cluster_plot(samples[1250])
cluster_plot(samples[2750])
#cluster_plot(samples[1000])
#cluster_plot(samples[4000])
samples_histogram(samples)


In [ ]:
k_cluster = [cluster for cluster in samples.values() if len(cluster) == 10]

In [ ]:
cluster_plot(k_cluster[5])

In [ ]:
k_max = 10

In [ ]:
def compute_mean_positions(samples, k_max=k_max):
    
    k_cluster = [cluster for cluster in samples.values() if len(cluster) == k_max]
    mean_positions = {i: [] for i in range(k_max)}

    def calculate_centroid(points):
        return np.mean(points, axis=0)

    for cluster in k_cluster:
        # Compute centroids for all clusters
        centroids = {k: calculate_centroid(v) for k, v in cluster.items()}
        sorted_keys = sorted(centroids.keys(), key=lambda k: (centroids[k][0], centroids[k][1]))
        
        for i, key in enumerate(sorted_keys):
            mean_positions[i].extend(cluster[key])

    mean_positions_result = {}
    for i in range(k_max):
        if len(mean_positions[i]) > 0:
            mean_positions_result[i] = np.mean(mean_positions[i], axis=0)
        else:
            mean_positions_result[i] = np.zeros(len(mean_positions[i][0]))
    return mean_positions_result

def plot_mean_positions(mean_positions):
    plt.figure(figsize=(10, 6), dpi=300)
    
    for i in range(len(mean_positions)):
        position = mean_positions[i]
        plt.scatter(position[0], position[1], label=f'Cluster {i}', s=100)
    
    # Plot true cluster centers
    for i, center in enumerate(true_cluster_centers):
        plt.scatter(center[0], center[1], s=100, marker='x', color='black')
    
    plt.xlabel('$x$ coordinate')
    plt.ylabel('$y$ coordinate')
    plt.title('Estimated mean positions $(\widehat{m}_\ell)_{1\leq \ell \leq |\mathcal{C}|}$ and $\\times$ true centroids')
    plt.legend()
    plt.grid(True)
    plt.show()

mean_positions = compute_mean_positions(samples, k_max)
print(mean_positions)
plot_mean_positions(mean_positions)

In [ ]:
list1 = list(mean_positions.values())
list2 = true_cluster_centers

import numpy as np

# Set k_max
inf = min(len(list1), len(list2))  # Ensure k_max does not exceed the length of the lists

list1 = list(mean_positions.values())[:inf]
list2 = true_cluster_centers[:inf]

# Calculate pairwise distances
distances = np.zeros((len(list1), len(list2)))
for i, point1 in enumerate(list1):
    for j, point2 in enumerate(list2):
        distances[i, j] = np.linalg.norm(point1 - point2)

# Start with the point whose distance to the second set is minimum
associations = []
used_indices_list2 = set()
for _ in range(len(list1)):
    min_distance = float('inf')
    min_index_list1 = -1
    min_index_list2 = -1
    
    for i in range(len(list1)):
        for j in range(len(list2)):
            if j in used_indices_list2:
                continue
            if distances[i, j] < min_distance:
                min_distance = distances[i, j]
                min_index_list1 = i
                min_index_list2 = j
                
    associations.append((min_index_list1, min_index_list2))
    used_indices_list2.add(min_index_list2)
    
# Compute the sum of the squared distances
sum_squared_distances = 0
for i, j in associations:
    dist = np.linalg.norm(list1[i] - list2[j])
    sum_squared_distances += dist ** 2
    print(f"Point {list1[i]} from list1 is closest to point {list2[j]} from list2 with squared distance {dist ** 2}")

# Calculate the mean of the squared distances and then the square root
mean_squared_distance = sum_squared_distances / len(associations)
sqrt_mean_squared_distance = np.sqrt(mean_squared_distance)

print(f"Square root of the mean of the squared distances: {sqrt_mean_squared_distance}")

